# Préparation des Données et Ingénierie des Caractéristiques

Les données sont cruciales pour entraîner et préparer un modèle. Dans ce notebook, nous allons couvrir comment charger des données dans ML.NET et s'assurer qu'elles sont dans le bon format pour que ML.NET puisse les utiliser.

## Chargement des données dans ML.NET

### Qu'est-ce qu'un IDataView?

Le [IDataView](https://docs.microsoft.com/dotnet/api/microsoft.ml.idataview?view=ml-dotnet) est le format de données que ML.NET charge pour l'entraînement. Il s'agit d'un ensemble d'interfaces et de composants qui fournissent un traitement efficace et compositionnel des données schématisées pour les applications de machine learning et d'analytique avancée. Il est conçu pour gérer de manière efficace les données de haute dimension et les grands ensembles de données.

### Comment créer un IDataView

Vous pouvez créer un IDataView en utilisant l'une des méthodes de chargement des données suivantes :

- TextLoader
- LoadFromEnumerable
- DatabaseLoader
- LoadFromTextFile

Pour plus de documentation et d'exemples, consultez [Charger les données à partir de fichiers et d'autres sources](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/load-data-ml-net).



In [ ]:
#r "nuget: Microsoft.ML"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

MLContext mlContext = new MLContext();

### Télécharger ou localiser les données

Le code suivant essaie de localiser le fichier de données dans quelques emplacements connus ou le téléchargera à partir de l'emplacement GitHub connu.

In [ ]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{
    // Ce chemin est utilisé si le dépôt a été cloné.
    var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

    if (!File.Exists(filePath))
    {
        // Ce chemin est utilisé si le fichier a déjà été téléchargé.
        filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
    }

    if (!File.Exists(filePath))
    {
        using (var client = new WebClient())
        {
            client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
        }
        Console.WriteLine($"Téléchargé {fileName} à : {filePath}");
    }
    else
    {
        Console.WriteLine($"{fileName} trouvé ici : {filePath}");
    }

    return filePath;
}

### Chargement depuis un fichier

Un [TextLoader](https://docs.microsoft.com/dotnet/api/microsoft.ml.data.textloader?view=ml-dotnet) peut charger un fichier structuré dans un IDataView. Les informations structurées sont représentées comme des colonnes et des lignes de données.


In [ ]:
public class ModelInput
{
    [LoadColumn(0)]
    [ColumnName(@"vendor_id")]
    public string Vendor_id { get; set; }

    [LoadColumn(1)]
    [ColumnName(@"rate_code")]
    public float Rate_code { get; set; }

    [LoadColumn(2)]
    [ColumnName(@"passenger_count")]
    public float Passenger_count { get; set; }

    [LoadColumn(3)]
    [ColumnName(@"trip_time_in_secs")]
    public float Trip_time_in_secs { get; set; }

    [LoadColumn(4)]
    [ColumnName(@"trip_distance")]
    public float Trip_distance { get; set; }

    [LoadColumn(5)]
    [ColumnName(@"payment_type")]
    public string Payment_type { get; set; }

    [LoadColumn(6)]
    [ColumnName(@"fare_amount")]
    public float Fare_amount { get; set; }
}

MLContext mlContext = new MLContext();

var trainDataPath = EnsureDataSetDownloaded("taxi-fare.csv");

// Créer un TextLoader basé sur le type ModelInput.
TextLoader textLoader = mlContext.Data.CreateTextLoader<ModelInput>(separatorChar: ',', hasHeader: true);

// Charger les données dans un IDataView. La méthode Load() peut prendre en charge plusieurs fichiers.
// Les fichiers doivent avoir le même caractère séparateur, en-tête, noms de colonnes, etc.
IDataView data = textLoader.Load(trainDataPath);

data.Preview(1)

## Chargement d'une collection en mémoire

ML.NET prend en charge le chargement des données à partir d'une collection en mémoire. Cela facilite le chargement à partir d'un fichier JSON ou XML en utilisant C#. Apprenez comment [désérialiser JSON avec C#](https://docs.microsoft.com/dotnet/standard/serialization/system-text-json-how-to?pivots=dotnet-6-0#how-to-read-json-as-net-objects-deserialize) ou utilisez [XML serializer](https://docs.microsoft.com/dotnet/api/system.xml.serialization?view=net-6.0) pour obtenir ces fichiers en mémoire.

Une fois que vous avez la collection de données en mémoire, vous pouvez la charger dans ML.NET avec la méthode [LoadFromEnumerable](https://docs.microsoft.com/dotnet/api/microsoft.ml.dataoperationscatalog.loadfromenumerable?view=ml-dotnet).


In [ ]:
ModelInput[] inMemoryCollection = new ModelInput[]
{
    new ModelInput
    {
        Vendor_id = "CMT",
        Rate_code = 1,
        Passenger_count = 1,
        Trip_time_in_secs = 1271,
        Trip_distance = 3.8f,
        Payment_type = "CRD",
        Fare_amount = 17.5f,
    },
    new ModelInput
    {
        Vendor_id = "VST",
        // Rate_code manquant
        Passenger_count = 1,
        Trip_time_in_secs = 474,
        Trip_distance = 1.5f,
        Payment_type = "CSH",
        Fare_amount = 8,
    }
};

// Créer un MLContext
MLContext mlContext = new MLContext();

// Charger les données
IDataView data = mlContext.Data.LoadFromEnumerable<ModelInput>(inMemoryCollection);

// Afficher les données chargées
var preview = data.Preview();
display(preview);

### Différence entre DataFrame et IDataView

Vous avez peut-être entendu parler du type [DataFrame](https://docs.microsoft.com/dotnet/api/microsoft.data.analysis.dataframe?view=ml-dotnet-preview). C'est un autre outil pour charger, visualiser et manipuler des données, courant dans les notebooks. Il implémente un IDataView, il peut donc être facilement passé à ML.NET.

DataFrame et IDataView sont très similaires en ce sens qu'ils sont tous deux des moyens de représenter des données sous forme tabulaire et d'appliquer des transformations. Voici quelques différences clés :

- DataFrame ne prend en charge que le chargement de fichiers délimités.
- DataFrame fonctionne en mémoire, vous êtes donc limité par la quantité de mémoire de votre PC.

Le DataFrame est recommandé pour effectuer des tâches telles que l'analyse exploratoire des données sur un échantillon de vos données. Consultez le notebook de référence [REF-Data Processing](https://github.com/dotnet/csharp-notebooks/blob/main/machine-learning/REF-Data%20Processing.ipynb) pour un exemple d'utilisation des DataFrames pour manipuler un fichier de données pour l'entraînement.

IDataView est recommandé pour l'entraînement sur des ensembles de données plus grands, et ce qui est utilisé pour les exemples dans ce notebook. Les ensembles de données plus grands dans ce cas sont définis comme des ensembles de données qui ne peuvent pas tenir en mémoire.

## Transformations des données

ML.NET prend en charge une variété de transformations de données qui convertiront les données dans le format requis et vous aideront à apporter des corrections à vos données. Parmi les opérations courantes, citons la manipulation des colonnes, la normalisation des valeurs, le remplacement des valeurs manquantes, la conversion des valeurs, etc.

Pour plus d'informations, consultez [Transformations des données](https://docs.microsoft.com/dotnet/machine-learning/resources/transforms).

### Données catégorielles

L'encodage one-hot est une transformation importante pour les données contenant des catégories. Les algorithmes ML nécessitent des données numériques, ils ne savent pas comment traiter les chaînes représentant des catégories. Les colonnes vendor_id et payment_type sont catégoriques, le fournisseur peut être "CMD" ou "VST" et le paiement peut être "CReDit" ou "CaSH". L'encodage one-hot prend les valeurs de chaîne passées et les convertit en données numériques.

In [ ]:
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding(
    new[] 
    { new InputOutputColumnPair(@"vendor_id"), 
    new InputOutputColumnPair(@"payment_type")},
    OneHotEncodingEstimator.OutputKind.Binary);

Testons la transformation ci-dessus sur les colonnes vendor_id et payment_type. Le résultat devrait être une valeur vectorielle pour chaque catégorie. Dans le cas de Vendor_Id, CMT devient `000` et VST devient `001`. Nous allons créer une nouvelle classe ModelInputTransformed pour les nouveaux types convertis.

In [ ]:
using System.Numerics;

public class ModelInputTransformed
{
    [LoadColumn(0)]
    [ColumnName(@"vendor_id")]
    public VBuffer<Single> Vendor_id { get; set; }

    [LoadColumn(1)]
    [ColumnName(@"rate_code")]
    public float Rate_code { get; set

; }

    [LoadColumn(2)]
    [ColumnName(@"passenger_count")]
    public float Passenger_count { get; set; }

    [LoadColumn(3)]
    [ColumnName(@"trip_time_in_secs")]
    public float Trip_time_in_secs { get; set; }

    [LoadColumn(4)]
    [ColumnName(@"trip_distance")]
    public float Trip_distance { get; set; }

    [LoadColumn(5)]
    [ColumnName(@"payment_type")]
    public VBuffer<Single> Payment_type { get; set; }

    [LoadColumn(6)]
    [ColumnName(@"fare_amount")]
    public float Fare_amount { get; set; }
}

// Exécuter la transformation
IDataView transformedData = pipeline.Fit(data).Transform(data);
var convertedData = mlContext.Data.CreateEnumerable<ModelInputTransformed>(transformedData, true);

// Encodage one-hot de deux colonnes 'vendor_id' et 'payment_type'.
Console.WriteLine("Vendor_Id" +"\t" + "Payment_Type"); 
foreach (ModelInputTransformed item in convertedData.Take(10))
{    
    Console.WriteLine("{0}\t\t{1}", string.Join(" ", item.Vendor_id.DenseValues()),
                    string.Join(" ", item.Payment_type.DenseValues()));
}

### Remplacer les valeurs manquantes

Une autre opération courante est de remplacer les valeurs manquantes. Ici, nous utilisons le mode de remplacement par défaut, qui remplace la valeur par la valeur par défaut de son type.


In [ ]:
pipeline.Append(mlContext.Transforms.ReplaceMissingValues(
    new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), 
    new InputOutputColumnPair(@"passenger_count", @"passenger_count"), 
    new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), 
    new InputOutputColumnPair(@"trip_distance", @"trip_distance") }));


Encore une fois, exécutons la transformation et jetons un coup d'œil à la valeur remplie. Il nous manquait le rate_code pour le deuxième objet factice.

In [ ]:
IDataView  transformedData = pipeline.Fit(data).Transform(data);
var convertedData = mlContext.Data.CreateEnumerable<ModelInputTransformed>(transformedData, true);

"Rate_code: " + convertedData.ElementAt(1).Rate_code

### Concaténer les colonnes de caractéristiques

Concaténons maintenant toutes nos colonnes de caractéristiques en une seule colonne vectorielle. De nombreux formateurs ML s'attendent à ce que les caractéristiques soient de type vecteur, car l'application d'opérations sur un vecteur est plus efficace.

In [ ]:
pipeline.Append(mlContext.Transforms.Concatenate(
    @"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

Nous avons maintenant un IDataView chargé et un pipeline prêt pour l'entraînement.